# Dropbox OAuth2 Credentials Notebook
This notebook implements the OAuth2 flow for Dropbox to download the credentials needed to access your Dropbox account.

In [ ]:
from dropbox.oauth import DropboxOAuth2FlowNoRedirect
import json

## Step 1
Get the credentials for your dropbox app. The [tutorial in section 1.4](https://github.com/AjayACST/gnss-ir-nz/blob/main/tutorial_pdfindex.pdf) has instructions on how to get your app_key and app_secret, paste them below.

Copy the link generated into a new tab in your browser, and paste the authorization code that you get back into the input below.

In [ ]:
app_key = 'your_app_key_here'
app_secret = 'your_app_secret_here'

flow = DropboxOAuth2FlowNoRedirect(
    consumer_key=app_key,
    consumer_secret=app_secret,
    token_access_type='offline',
)

authorize_url = flow.start()

print("\n1. Go to: {}".format(authorize_url))
print("2. Click \"Allow\" (you might have to log in first).")
print("3. Copy the authorization code.\n")

auth_code = input("Enter the authorization code here: ").strip()

try:
    oauth_result = flow.finish(auth_code)
except Exception as e:
    print("Error: {}".format(e))
    exit(1)

creds = {
    "access_token": oauth_result.access_token,
    "refresh_token": oauth_result.refresh_token,
    "expiration": oauth_result.expires_at.strftime("%m/%d/%Y, %H:%M:%S"),
}

out_path = 'dropbox_creds.json'

with open(out_path, 'w') as f:
    json.dump(creds, f)
print(f"Credentials saved to {out_path}")

If you are using these credentials for the arduino, right click the new file in the file browser to the left and select "Download", the tutorial PDF has further instructions for next steps.